In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-14 18:55:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-11-14 18:56:00 (1.05 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NCAA").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Extraction

In [4]:
from pyspark import SparkFiles
# Connecting to S3 for Basketball Stats
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NBA.csv"
spark.sparkContext.addFile(url)

NBAstats = spark.read.option("header", "true").csv(SparkFiles.get("NBA.csv"), inferSchema=True, sep=",")
# NBAstats.show(10)

In [5]:
NBAstats.count()

11762

In [ ]:
# Print our schema
NBAstats.printSchema()

In [6]:
from pyspark import SparkFiles
# Connecting to S3 for NBA draft data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NBAdraft.csv"
spark.sparkContext.addFile(url)

NBAdraft = spark.read.option("header", "true").csv(SparkFiles.get("NBAdraft.csv"), inferSchema=True, sep=",")
# NBAdraft.show(10)

In [7]:
NBAdraft.count()

300

In [ ]:
# Print our schema
NBAdraft.printSchema()

In [15]:
from pyspark import SparkFiles
# Connecting to S3 for NFL Draft Data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NFLdraft.csv"
spark.sparkContext.addFile(url)

NFL = spark.read.option("header", "true").csv(SparkFiles.get("NFLdraft.csv"), inferSchema=True, sep=",")
NFL.show(10)

+---------+---+----+--------------------+----------------+-----------+--------+--------------+---------------------+--------------+
|DraftYear|Rnd|Pick|                Team|          Player|GradeFileID|Position|       College|CollegeBudgetFileName|    Conference|
+---------+---+----+--------------------+----------------+-----------+--------+--------------+---------------------+--------------+
|     2019|  2|  51|    Tennessee Titans|      A.J. Brown|      46204|      WR|   Mississippi| University of Mis...|           SEC|
|     2015|  3|  67|Jacksonville Jaguars|       A.J. Cann|       9500|       G|South Carolina| University of Sou...|           SEC|
|     2015|  6| 202|New England Patriots|      A.J. Derby|       9635|      TE|      Arkansas| University of Ark...|           SEC|
|     2020|  2|  62|   Green Bay Packers|     A.J. Dillon|      57165|      RB|   Boston Col.|       Boston College|           ACC|
|     2020|  2|  54|       Buffalo Bills|    A.J. Epenesa|      56369|      

In [16]:
NFL.count()

1531

In [17]:
# Print our schema
NFL.printSchema()

root
 |-- DraftYear: integer (nullable = true)
 |-- Rnd: integer (nullable = true)
 |-- Pick: integer (nullable = true)
 |-- Team: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- GradeFileID: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- College: string (nullable = true)
 |-- CollegeBudgetFileName: string (nullable = true)
 |-- Conference: string (nullable = true)



In [ ]:
from pyspark import SparkFiles
# Connecting to S3 for School Data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/SchoolCleaned.csv"
spark.sparkContext.addFile(url)

School_df = spark.read.option("header", "true").csv(SparkFiles.get("SchoolCleaned.csv"), inferSchema=True, sep=",")
School_df.show(10)

In [ ]:
School_df.count()

In [ ]:
# Print our schema
School_df.printSchema()

**Transformation**

In [8]:
pdf = NBAstats.toPandas()
df_dedupe = pdf.rename(columns={'FG%':'FG_pct', '3PM':'ThreePM',
                                '3PA':'ThreePA', '3P%':'Three_pct',
                                'FT%':'FT_pct'}).drop_duplicates(subset=['Player'], keep='last').round(3)
NBAstats = spark.createDataFrame(df_dedupe)
NBAstats.show()

+-----------------+----+---+----+---+----+------+-------+-------+---------+---+---+------+---+---+---+---+---+---+---+---+----+
|           Player|Team| GP| MPG|FGM| FGA|FG_pct|ThreePM|ThreePA|Three_pct|FTM|FTA|FT_pct|TOV| PF|ORB|DRB|RPG|APG|SPG|BPG| PPG|
+-----------------+----+---+----+---+----+------+-------+-------+---------+---+---+------+---+---+---+---+---+---+---+---+----+
|     Tyler Harvey| EWU| 32|36.9|7.2|15.3| 0.469|    4.0|    9.3|    0.431|4.7|5.5| 0.852|2.0|1.9|0.6|3.0|3.6|2.6|1.1|0.1|23.1|
|       Tyler Haws| BYU| 35|32.1|7.4|15.4|  0.48|    1.5|    4.2|    0.367|5.9|6.7| 0.873|2.0|1.4|1.3|3.1|4.5|2.4|0.9|0.3|22.2|
|Denzel Livingston| UIW| 29|36.9|6.6|14.4| 0.455|    1.6|    4.6|    0.343|6.8|8.5| 0.805|2.8|3.0|1.5|4.4|5.9|2.9|2.6|1.3|21.5|
|      Saah Nimley| CSU| 31|35.7|5.9|15.4| 0.384|    3.5|    9.9|    0.358|6.0|7.4|  0.82|2.7|2.2|0.5|3.5|4.1|4.1|0.9|0.0|21.4|
|        Amere May|DESU| 32|34.2|7.1|17.0| 0.418|    2.8|    7.2|    0.387|4.0|4.9| 0.821|2.9|1.8|1.0|3.

In [9]:
NBAstats.count()

6508

In [10]:
pdf = NBAdraft.toPandas()
df_dedupe = pdf.rename(columns={'Pk':'Pick','Player':'Name'}).drop_duplicates(subset=['Name'], keep='last')
NBAdraft = spark.createDataFrame(df_dedupe)
NBAdraft.show()

+----+--------------------+--------------+--------------+
|Pick|                Name|    Conference|       College|
+----+--------------------+--------------+--------------+
|   8|       Collin Sexton|  Southeastern|       Alabama|
|   1|       Deandre Ayton|        PAC-12|       Arizona|
|   7|     Lauri Markkanen|        PAC-12|       Arizona|
|  53|        Kadeem Allen|        PAC-12|       Arizona|
|   8|     Stanley Johnson|        PAC-12|       Arizona|
|  23|Rondae Hollis-Jef...|        PAC-12|       Arizona|
|  38|      Daniel Gafford|  Southeastern|      Arkansas|
|  22|        Bobby Portis|  Southeastern|      Arkansas|
|  16|         Chuma Okeke|  Southeastern|        Auburn|
|  12|Taurean Waller-Pr...|        Big-12|        Baylor|
|  26|       Dylan Windler|          null|       Belmont|
|  22|  Chandler Hutchison| Mountain West|   Boise State|
|  13|     Jerome Robinson|Atlantic Coast|Boston College|
|  42|      Olivier Hanlan|Atlantic Coast|Boston College|
|  37|      Ri

In [11]:
NBAdraft.count()

299

In [ ]:
School_df = School_df.withColumnRenamed("REV_MEN", "Revenue")
School_df = School_df.withColumnRenamed("EXP_MEN", "Expense")
School_df.show()

**Loading**

In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ncaa-athletics.cr5bt5kg46tf.us-west-1.rds.amazonaws.com:5432/NCAA_Athletics"
config = {"user":"postgres", 
          "password": "group1final", 
          "driver":"org.postgresql.Driver"}

In [13]:
# Writing NBA Stats table into RDS
NBAstats.write.jdbc(url=jdbc_url, table='BballStats', mode=mode, properties=config)

In [14]:
# Writing NBA draft data into RDS
NBAdraft.write.jdbc(url=jdbc_url, table='NBAdraft', mode=mode, properties=config)

In [20]:
# Writing NFL draft data into RDS
NFL.write.jdbc(url=jdbc_url, table='NFL', mode=mode, properties=config)

In [ ]:
# Writing Schools table into RDS
School_df.write.jdbc(url=jdbc_url, table='schools', mode=mode, properties=config)